# Test Audio Sender

This file is used for sending audio data to the device. It measures the accuracy based on individual audio files.

**Note that the software on the MCU must match this!**

In [145]:
AUDIO_FOLDER_PATH = "dataset_mix"
POSITIVE_PATH = f"{AUDIO_FOLDER_PATH}/positive"
NEGATIVE_PATH = f"{AUDIO_FOLDER_PATH}/negative"

In [146]:
import numpy as np
import pandas as pd
import gc
import os
import librosa
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import serial
import struct

In [147]:
# Collecte positive audio files and their classes
positive_audio_files = []
positive_audio_file_classes = []
# Look each file in the audio folder
audio_class_folders = os.listdir(POSITIVE_PATH)
# Loop each class folder
for audio_class_folder in audio_class_folders:
    # Assemble full audio class folder path.
    audio_class_folder_path = os.path.join(POSITIVE_PATH, audio_class_folder)
    print("Processing class folder: ", audio_class_folder_path)
    # Get all files in the audio class folder
    audio_class_files = os.listdir(audio_class_folder_path)
    # Loop each audio file in the audio class folder
    for audio_class_file in audio_class_files:
        # Assemble full audio file path.
        audio_file_path = os.path.join(audio_class_folder_path, audio_class_file)
        # Append the audio file path to the positive_audio_files list
        positive_audio_files.append(audio_file_path)
        # Append the audio class to the audio_file_classes list
        positive_audio_file_classes.append(audio_class_folder)
print("positive_audio_files length: ", len(positive_audio_files))
print("positive_audio_file_classes length: ", len(positive_audio_file_classes))

Processing class folder:  dataset_mix/positive/Car
Processing class folder:  dataset_mix/positive/Comm
Processing class folder:  dataset_mix/positive/Motorcycle
positive_audio_files length:  18597
positive_audio_file_classes length:  18597


In [148]:
positive_audio_file_classes[0]

'Car'

In [149]:
# Collecte negative audio files and their classes
negative_audio_files = []
negative_audio_file_classes = []
# Look each file in the audio folder
audio_class_folders = os.listdir(NEGATIVE_PATH)
# Loop each class folder
for audio_class_folder in audio_class_folders:
    # Assemble full audio class folder path.
    audio_class_folder_path = os.path.join(NEGATIVE_PATH, audio_class_folder)
    print("Processing class folder: ", audio_class_folder_path)
    # Get all files in the audio class folder
    audio_class_files = os.listdir(audio_class_folder_path)
    # Loop each audio file in the audio class folder
    for audio_class_file in audio_class_files:
        # Assemble full audio file path.
        audio_file_path = os.path.join(audio_class_folder_path, audio_class_file)
        # Append the audio file path to the negative_audio_files list
        negative_audio_files.append(audio_file_path)
        # Append the audio class to the audio_file_classes list
        negative_audio_file_classes.append(audio_class_folder)
print("negative_audio_files length: ", len(negative_audio_files[0:10]))
print("negative_audio_file_classes length: ", len(negative_audio_file_classes[0:10]))

Processing class folder:  dataset_mix/negative/background
negative_audio_files length:  10
negative_audio_file_classes length:  10


In [150]:
# Shuffle the audio files and classes with the same seed.
seed = 42
np.random.seed(seed)
np.random.shuffle(positive_audio_files)

np.random.seed(seed)
np.random.shuffle(positive_audio_file_classes)

np.random.seed(seed)
np.random.shuffle(negative_audio_files)

In [151]:
# Hot end code the labels.
label_encoder = LabelEncoder()
positive_audio_file_classes_categorical = to_categorical(label_encoder.fit_transform(positive_audio_file_classes))
print("Example of audio_file_classes_categorial: ", positive_audio_file_classes_categorical[0:10])

Example of audio_file_classes_categorial:  [[0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [152]:
# Connect to MCU via serial.
print("Configuring serial port...")
ser = serial.Serial(
    port='/dev/ttyACM0',  # Change this to your actual port, e.g., 'COM3' on Windows, '/dev/ttyS0' on Linux
    baudrate=921600,       # Set baud rate to 921600
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    timeout=0.1,           # Set timeout for reading
    write_timeout=None     # Wait indefinitely until all data is sent
)

Configuring serial port...


In [153]:
if ser.is_open:
    print(f"Serial port {ser.port} opened at {ser.baudrate} baud.")

Serial port /dev/ttyACM0 opened at 921600 baud.


In [154]:
def calc_accuracy(realResults, results):
    correct = 0
    total = min(len(realResults), len(results))  # Ensure we only compare up to the shorter length
    for real, predicted in zip(realResults[:total], results[:total]):
        if real == predicted:
            correct += 1
    return correct / total if total > 0 else 0.0

import numpy as np

def confusion_matrix_manual(y_true, y_pred):

    labels = sorted(set(y_true + y_pred))  # All unique labels
    matrix = {label: {l: 0 for l in labels} for label in labels}

    for real, pred in zip(y_true, y_pred):
        matrix[real][pred] += 1

    return matrix


def print_confusion_matrix(cm):
    labels = list(cm.keys())
    print("    " + " ".join(f"{l:>4}" for l in labels))
    for real in labels:
        row = " ".join(f"{cm[real][pred]:>4}" for pred in labels)
        print(f"{real:>4} {row}")

In [155]:
audioSent = 0
audioFilesProcessed = 0
results = []
realResults = []

In [156]:
positive_audio_file_classes_categorical[0]

array([0., 1., 0.])

In [157]:
def streamAudioFile(file_path, realClass, i):
    global audioSent
    global results
    global realResults
    global audioBuffer

    # Append the real result to the results list.
    realResults.append(np.argmax(realClass))

    # Check file format.
    if not file_path.endswith('.wav'):
        print("Error: Only .wav files are supported.")
        return

    # Load the audio file.
    audio_data, sr = librosa.load(file_path, sr=None)

    # Convert sample rate to 16kHz.
    if sr != 16000:
        audio_data = librosa.resample(y=audio_data, orig_sr=sr, target_sr=16000)
        sr = 16000

    # Convert the audio data to a numpy array.
    audio_data = np.array(audio_data, dtype=np.float32)

    # Send the audio data to the MCU.
    ser.write(audio_data.tobytes())

    classifications = []

    while ser.in_waiting > 0:
        response = ser.readline().decode('utf-8').strip()
        if response.startswith("fin:"):
            matrix = confusion_matrix_manual(realResults, results)
            print_confusion_matrix(matrix)
        if response.startswith("e:"):
            print(response)
            return
        if response.startswith("c:"):
            print(response)
        if response.startswith("s:"):
            print(response)
        if response.startswith("v:"):
            print(response)
            response = int(response[2:])
            classifications.append(response)

    replaced = False
    if len(results) > 0:
        if results[-1] == 9:
            if (len(classifications) > 0):
                results[-1] = classifications.pop(0)
                replaced = True

    if len(classifications) > 0:
        results.append(classifications.pop(0))
        if replaced:
            realResults.append(realResults[-1])

    while len(classifications) > 0:
        results.append(classifications.pop(0))
        realResults.append(realResults[-1])
    
    # If no classification was received, append None to results.
    if i > 0 and len(results) < len(realResults):
        results.append(9)
    print(f"Accuracy: {calc_accuracy(realResults, results)}")

In [ ]:

total_positive_audio_files = len(positive_audio_files)

# Loop each audio file.
negative_audio_file_pointer = 0
for i in range(500):
    # Get audio file path.
    audio_file = positive_audio_files[i+65]
    # Print audio file path and class.
    print("Processing audio file: ", audio_file)
    print("Processing audio file class: ", positive_audio_file_classes[i+65])
    # Stream positive audio file to MCU.
    streamAudioFile(audio_file, positive_audio_file_classes_categorical[i+65], i)
    # Print nagtive audio file path and class.
    print("Processing negative audio file: ", negative_audio_files[negative_audio_file_pointer])
    # Stream negative audio file to MCU.
    streamAudioFile(negative_audio_files[negative_audio_file_pointer], [0,0,0,1], i)
    # Increment the negative audio file pointer.
    negative_audio_file_pointer = (negative_audio_file_pointer + 1) % len(negative_audio_files)

Processing audio file:  dataset_mix/positive/Comm/2019-11-19-07-25_Langewiesener-Strasse_50Kmh_2055128_M_D_TR_SE_CH34.wav
Processing audio file class:  Comm
c: [57,-85,-114,-114] voted for: 0 max value: 57
c: [111,-165,-230,-228] voted for: 0 max value: 54
c: [197,-277,-345,-342] voted for: 0 max value: 86
c: [280,-386,-460,-457] voted for: 0 max value: 83
c: [362,-495,-575,-570] voted for: 0 max value: 82
c: [434,-595,-689,-684] voted for: 0 max value: 72
c: [511,-699,-803,-799] voted for: 0 max value: 77
Accuracy: 0.0
Processing negative audio file:  dataset_mix/negative/background/2019-11-13-15-55_Schleusinger-Allee_70Kmh_124454655_ME_CH12-BG.wav
c: [585,-801,-916,-914] voted for: 0 max value: 74
c: [475,-908,-1029,-840] voted for: 3 max value: 74
c: [361,-1022,-1144,-753] voted for: 3 max value: 87
c: [248,-1135,-1257,-671] voted for: 3 max value: 82
v:0
c: [-299,-639,-682,83] voted for: 3 max value: 70
c: [-409,-749,-794,160] voted for: 3 max value: 77
c: [-485,-759,-897,93] voted

In [ ]:
print(len(results))
print(len(realResults))

85
84


In [ ]:
print(results)
print(list(map(int, realResults)))

#[1, 3, 1, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 2, 3, 1, 3, 1, 3, 0, 3, 0, 3, 0, 3, 9, 9, 2, 9, 3, 0, 3, 1, 3, 0, 3, 1, 3, 1, 3, 1, 3, 0, 3, 0, 3, 0, 3, 0]
#[1, 3, 1, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 1, 3, 0, 3, 0, 3, 2, 3, 1, 3, 1, 3, 0, 3, 0, 3, 0, 3, 0, 3, 2, 3, 0, 3, 1, 3, 0, 3, 1, 3, 1, 3, 1, 3, 0, 3, 0, 3, 0, 3, 0, 3, 1]

[0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 2, 3, 1, 3, 0, 3, 0, 3, 2, 3, 2, 3, 0, 3, 1, 3, 0, 3, 2, 3, 0, 3, 1, 3, 2, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 2, 3, 0, 3, 0, 3, 0, 3, 2, 3, 2, 3, 0, 3, 0, 3, 0, 3, 1, 3, 0, 3, 1, 3, 0, 3, 2, 3, 2, 3, 0, 3, 0, 3, 0]
[1, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 2, 3, 1, 3, 0, 3, 0, 3, 2, 3, 0, 3, 1, 3, 0, 3, 2, 3, 0, 3, 1, 3, 2, 3, 0, 3, 0, 3, 0, 3, 2, 3, 0, 3, 2, 3, 0, 3, 1, 3, 0, 3, 2, 3, 2, 3, 0, 3, 1, 3, 0, 3, 1, 3, 0, 3, 1, 3, 0, 3, 2, 9, 3, 0, 3, 0, 3, 0, 3, 1]


In [ ]:
# Close the serial port
ser.close()